In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
print( sys.path )

In [ ]:
### Make a list of files
import yaml
#from box import Box


with open('configure_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)


######################################################

x='oldCTL'#'alpha04ax010'
#x='ne240x2' #_QxQsst'
#x='ne240x2_L56' #_QxQsst'

exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
ymdPat = cfg[x]['ymdPat']
season = cfg['plot']['season']
valData  = cfg['validation']['data']
valYears = cfg['validation']['years']
print( exp, subd, Src, Hkey, Dst, useri )
print( ymdPat , season )
print( valData , valYears )


In [ ]:
%%time

A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )

exp_Val='ERA5'

In [ ]:
########################################
# Make seasonal means
########################################

print( f'{A.exp} Years {len(A.X.time) /12. }')

UUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='U',return_time=True)
TTa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='T',return_time=True)


In [ ]:
########################################
# Make zonal means
########################################
method='C2'

if (Hkey == 'yx' ):
    ########################
    # Option 1 from lat-lon regirrdded data
    ########################
    
    
    
    #ps_x =Dx.PS.values
    lon_a=A.X.lon.values
    lat_a=A.X.lat.values
    lev_a=A.X.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )
    # Zonal Av
    UUa_sz=np.average( UUa_s , axis=2 )
    TTa_sz=np.average( TTa_s , axis=2 )

elif (Hkey=='c'):
    ########################
    # Option 2 from unstructured SE data
    ########################
    
    Dst = 'fv1x1'
    DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
    lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
    lev_a=A.X.lev.values
    zlev_a=-7.0*np.log( lev_a/1_000. )

    try:
        RegridObj
    except NameError:
        print("Regirdding Object will be created !!!! ")
        RegridObj = RgF.Horz(Src=Src, Dst=Dst )
    else:
        print("..... Rergridding Object has been created")

    UUa_s_xy=RgF.Horz(xfld_Src=UUa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    TTa_s_xy=RgF.Horz(xfld_Src=TTa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
    
    # Zonal Av
    UUa_sz=np.average( UUa_s_xy , axis=2 )
    TTa_sz=np.average( TTa_s_xy , axis=2 )


In [ ]:
DateRange = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False


valYears='*' #'2004'
Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
UUv_s , zlev_v,lat_v,lon_v, years_Uv = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon'], Uval['years']
UUv_sz=np.average( UUv_s , axis=2 )

Tval = Val.data(fld='T',season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
TTv_s , zlev_v,lat_v,lon_v, years_Tv = Tval['aa'], Tval['lev'], Tval['lat'], Tval['lon'], Tval['years']
TTv_sz=np.average( TTv_s , axis=2 )



In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))


# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=2,2
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(UUa_sz)
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 12))


ulev=np.linspace( -60,140,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=18

"""
titles = [ f"Control: $\Delta x$=100km (ne30) {season.upper()}" ,
          f"$\Delta x$=28km (ne120) {season.upper()} " ,
          f"$\Delta x$=14km (ne240) {season.upper()} " ,
          f"Validation <{exp_Val}> {season.upper()}  " ]
"""
titles = [ f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" ,
          f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" ,
          f"<{valData}> {season.upper()} {years_Uv} " ,
          f"<{valData}> {season.upper()} {years_Tv} " ]
plotTop=85

n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,UUa_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,UUa_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)


n=3
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

n=2
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,TTa_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,TTa_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)


n=4
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, TTv_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, TTv_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')


FigName = f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
print( FigName) 
Pu.save_link(png_filename = FigName )

